In [1]:
# import library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df = pd.read_csv('insurance.csv')

In [3]:
df.head(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
0,67,119.8,1.56,2.92,False,Jaipur,retired,High
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium
4,69,62.2,1.60,3.94,True,Indore,retired,High


In [4]:
df.shape

(100, 8)

In [5]:
df.isnull().sum()

age                           0
weight                        0
height                        0
income_lpa                    0
smoker                        0
city                          0
occupation                    0
insurance_premium_category    0
dtype: int64

In [6]:
df['occupation'].unique()

array(['retired', 'freelancer', 'student', 'government_job',
       'business_owner', 'unemployed', 'private_job'], dtype=object)

## Feature Engineering

In [7]:
# Feature 1: BMI
df["bmi"] = df["weight"] / (df["height"] ** 2)

In [8]:
df.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875


In [9]:
# Feature 2: Age Group
def age_group(age):
    if age < 25:
        return "young"
    elif age < 45:
        return "adult"
    elif age < 60:
        return "middle_aged"
    return "senior"

In [10]:
df["age_group"] = df["age"].apply(age_group)

In [11]:
df.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,age_group
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482,senior
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017,adult
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382,adult
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900,young
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875,senior


In [12]:
# Feature 3: Lifestyle Risk
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] or row["bmi"] > 27:
        return "medium"
    else:
        return "low"

In [13]:
df["lifestyle_risk"] = df.apply(lifestyle_risk, axis=1)

In [14]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"]

In [15]:
# Feature 4: City Tier
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [16]:
df["city_tier"] = df["city"].apply(city_tier)

In [17]:
df.head()

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category,bmi,age_group,lifestyle_risk,city_tier
0,67,119.8,1.56,2.92,False,Jaipur,retired,High,49.227482,senior,medium,2
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low,30.189017,adult,medium,1
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low,21.118382,adult,low,2
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium,45.535900,young,high,1
4,69,62.2,1.60,3.94,True,Indore,retired,High,24.296875,senior,medium,2


In [18]:
df.drop(columns=['age', 'weight', 'height', 'smoker', 'city'], inplace=True)

In [19]:
df.head()

,income_lpa,occupation,insurance_premium_category,bmi,age_group,lifestyle_risk,city_tier
0,2.92,retired,High,49.227482,senior,medium,2
1,34.28,freelancer,Low,30.189017,adult,medium,1
2,36.64,freelancer,Low,21.118382,adult,low,2
3,3.34,student,Medium,45.535900,young,high,1
4,3.94,retired,High,24.296875,senior,medium,2


In [20]:
# Select imput and output features
X = df[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df["insurance_premium_category"]

In [21]:
X.head()

,bmi,age_group,lifestyle_risk,city_tier,income_lpa,occupation
0,49.227482,senior,medium,2,2.92,retired
1,30.189017,adult,medium,1,34.28,freelancer
2,21.118382,adult,low,2,36.64,freelancer
3,45.535900,young,high,1,3.34,student
4,24.296875,senior,medium,2,3.94,retired


In [22]:
y.head()

0      High
1       Low
2       Low
3    Medium
4      High
Name: insurance_premium_category, dtype: object

In [23]:
# Define categorical and numeric features
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

In [24]:
# Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

In [25]:
# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [26]:
# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [27]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.9

In [28]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
    pickle.dump(pipeline, f)